## Creating an index and populating it with documents using Milvus and Nomic AI Embeddings

Simple example on how to ingest PDF documents, then web pages content into a Milvus VectorStore. . In this example, the embeddings are the fully open source ones released by NomicAI, [nomic-embed-text-v1](https://huggingface.co/nomic-ai/nomic-embed-text-v1).

As described in [this blog post](https://blog.nomic.ai/posts/nomic-embed-text-v1), those embeddings feature a "8192 context-length that outperforms OpenAI Ada-002 and text-embedding-3-small on both short and long context tasks". In additions, they are:

- Open source
- Open data
- Open training code
- Fully reproducible and auditable

Requirements:
- A Milvus instance, either standalone or cluster.

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 pymilvus==2.3.6 sentence-transformers==2.4.0 grpcio grpcio-reflection


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
import os
from langchain.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from caikit_nlp_client import LangchainEmbeddings

### Base parameters, the Milvus connection info

In [3]:
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "collection_nomicai_embeddings"

## Initial index creation and document ingestion

#### Download and load pdfs

In [4]:
product_version = 2.12
documents = [
    "release_notes",
    "introduction_to_red_hat_openshift_ai",
    "getting_started_with_red_hat_openshift_ai_self-managed",
    "openshift_ai_tutorial_-_fraud_detection_example",
    "developing_a_model",
    "integrating_data_from_amazon_s3",
    "working_on_data_science_projects",
    "serving_models",
    "monitoring_data_science_models",
    "managing_users",
    "managing_resources",
    "installing_and_uninstalling_openshift_ai_self-managed",
    "installing_and_uninstalling_openshift_ai_self-managed_in_a_disconnected_environment",
    "upgrading_openshift_ai_self-managed",
    "upgrading_openshift_ai_self-managed_in_a_disconnected_environment",   
]

pdfs = [f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/pdf/{doc}/red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us.pdf" for doc in documents]
pdfs_to_urls = {f"red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us": f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/html-single/{doc}/index" for doc in documents}

In [5]:
docs_dir = f"rhoai-doc-{product_version}"

if not os.path.exists(docs_dir):
    os.mkdir(docs_dir)
    for pdf in pdfs:
        try:
            response = requests.get(pdf)
        except:
            print(f"Skipped {pdf}")
            continue
        if response.status_code!=200:
            print(f"Skipped {pdf}")
            continue  
        with open(f"{docs_dir}/{pdf.split('/')[-1]}", 'wb') as f:
            f.write(response.content)
else:
    print('PDF dir found, skipping doc load')

PDF dir found, skipping doc load


In [6]:
pdf_folder_path = f"./rhoai-doc-{product_version}"

pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
pdf_docs = pdf_loader.load()

#### Inject metadata

In [7]:
from pathlib import Path

for doc in pdf_docs:
    doc.metadata["source"] = pdfs_to_urls[Path(doc.metadata["source"]).stem]

#### Load websites

In [8]:
websites = [
    "https://ai-on-openshift.io/getting-started/openshift/",
    "https://ai-on-openshift.io/getting-started/opendatahub/",
    "https://ai-on-openshift.io/getting-started/openshift-ai/",
    "https://ai-on-openshift.io/odh-rhoai/configuration/",
    "https://ai-on-openshift.io/odh-rhoai/custom-notebooks/",
    "https://ai-on-openshift.io/odh-rhoai/nvidia-gpus/",
    "https://ai-on-openshift.io/odh-rhoai/custom-runtime-triton/",
    "https://ai-on-openshift.io/odh-rhoai/openshift-group-management/",
    "https://ai-on-openshift.io/tools-and-applications/minio/minio/",
    "https://access.redhat.com/articles/7047935",
    "https://access.redhat.com/articles/rhoai-supported-configs",
]

In [9]:
website_loader = WebBaseLoader(websites)
website_docs = website_loader.load()

#### Merge both types of docs

In [10]:
docs = pdf_docs + website_docs

#### Split documents into chunks with some overlap

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=126,
                                               chunk_overlap=40)
all_splits = text_splitter.split_documents(docs)
all_splits[0]

Document(page_content='Red Hat OpenShift AI Self-Managed\n \n2.12\nGetting started with Red Hat OpenShift AI\nSelf-Managed', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.12/html-single/getting_started_with_red_hat_openshift_ai_self-managed/index', 'page': 0})

#### Create the index and ingest the documents

In [12]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
#model_kwargs = {'trust_remote_code': True, 'device': 'cuda'}
#embeddings = HuggingFaceEmbeddings(
#    model_name="nomic-ai/nomic-embed-text-v1",
#    model_kwargs=model_kwargs,
#    show_progress=True
#)

#model = "flan-t5-small-caikit"
#model_endpoint_url = "https://caikit-flan-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com"
model = "all-MiniLM-L12-v2-caikit"
model_endpoint_url = "https://caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com"
token = '''\
Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6IjZnRXdDUGJfbE44Q29tTWNER0ZtRWN2RE52REt2MlRPTE95TE1USGRYTzQifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3
NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJqYXJ5LXdiIiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3
VudC9zZWNyZXQubmFtZSI6ImRlZmF1bHQtbmFtZS1jYWlraXQtZmxhbi1zYSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50Lm
5hbWUiOiJjYWlraXQtZmxhbi1zYSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50LnVpZCI6IjE2MGYzNTEzLTg4MTctNDRjMy
05NjMxLWVkMDY1MzA5Zjc4YiIsInN1YiI6InN5c3RlbTpzZXJ2aWNlYWNjb3VudDpqYXJ5LXdiOmNhaWtpdC1mbGFuLXNhIn0.XFYak7zyQYIp9Q5SOZlopkzTCj
Uei_zA9ITfZlpjkgrl74_tyH7DSl_AJIrMrjd689hjBNABU6tNJVlQGdGYljJzr2uYu57JujY2R6WO4X-pbvb6tyaG_nTq14DMzfgWZ4bT3hoH4elrQOAgMByT8k
d4iJexawExzTgfWiyCvKKK6eYdUFmO7-BQKK3WrFMDvfAzfK8PtHaJFnze0yZZgSGZMzixqHIUPAogZPwcZg-G1JTwzUiC2Wm5TmdfLXwLsDOApUsfn3_HFgL3Li
bNRPocgrjGMeh113j0O6m5dqw5RVrcRz_W8u1IycTOf-YEN2M2ak7HdJF5YB2i4ySK3RgCafXn5Re4kB2ieY0QNdvWOKo_ZPXpgwXA3iJ2P5s4fwq4g3xYvjCXIh
HaDGlqPl49p2L5skKdL6gBiJYNCBcfpT9su1ZKROIIQ-Auv02szvBkN3E3WBWX--GuMYEFjY5xla4JoVwVp0pwCatkysCgMopI8Fk5vLzm9I6LMK_uY8iUVHa_EZ
9hVBHBMkbXtoh-Icsm78uzcpuOfFEWx3DhTzr4Vau28jEiWwI2X0M5QcRJdHtb2rjczYfN8CxoAwVzAUZTAEihlVEsmcCy4ki-nFwfUKeYZIdJRK_YKGqkdKh8RL
WeeCY5tn-qtM3QXb_u42epG84SRqt-9BXT1iU
'''.replace('\n', '')

############################################################################################
# TODO: COMPARE MY NEW EMBEDDINGS TO HF ONE FOR MISSING PARAMS / DIFFS?
############################################################################################

embeddings = LangchainEmbeddings(
    token=token,
    endpoint=model_endpoint_url,
    model=model
)

db = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    auto_id=True,
    drop_old=True
    )

In [13]:
db.add_documents(all_splits[0:2])

/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[452551956821749113, 452551956821749114]

#### Alternatively, add new documents

In [14]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
# model_kwargs = {'trust_remote_code': True, 'device': 'cuda'}
# embeddings = HuggingFaceEmbeddings(
#     model_name="nomic-ai/nomic-embed-text-v1",
#     model_kwargs=model_kwargs,
#     show_progress=True
# )

# db = Milvus(
#     embedding_function=embeddings,
#     connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
#     collection_name=MILVUS_COLLECTION,
#     metadata_field="metadata",
#     text_field="page_content",
#     auto_id=True,
#     drop_old=False
#     )

# db.add_documents(all_splits)

#### Test query

In [15]:
query = "How can I work with GPU and taints in OpenShift AI?"
docs_with_score = db.similarity_search_with_score(query)

/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
RPC error: [search], <ParamError: (code=1, message=`search_data` value [dict_values([{'data': {'values': [0.014584872871637344, -0.06318628787994385, 0.04656320810317993, 0.0041893115267157555, 0.055654026567935944, -0.023691782727837563, -0.035480696707963943, -0.024649253115057945, 0.03860316798090935, -0.04051440954208374, -0.09852523356676102, -0.0668564885854721, -0.05555010586977005, -0.022397201508283615, 0.05034366250038147, -0.0031785545870661736, -0.002295487094670534, 0.06766505539417267, 0.00147917284630239, -0.008932314813137054, -0.01815999299287796, 0.008372127078473568, -0.005877046380192041, -0.0624219328165

single-end::<class 'dict_values'>


ParamError: <ParamError: (code=1, message=`search_data` value [dict_values([{'data': {'values': [0.014584872871637344, -0.06318628787994385, 0.04656320810317993, 0.0041893115267157555, 0.055654026567935944, -0.023691782727837563, -0.035480696707963943, -0.024649253115057945, 0.03860316798090935, -0.04051440954208374, -0.09852523356676102, -0.0668564885854721, -0.05555010586977005, -0.022397201508283615, 0.05034366250038147, -0.0031785545870661736, -0.002295487094670534, 0.06766505539417267, 0.00147917284630239, -0.008932314813137054, -0.01815999299287796, 0.008372127078473568, -0.005877046380192041, -0.06242193281650543, -0.11136991530656815, 0.06901022791862488, 0.020539313554763794, -0.0208711177110672, 0.06108584254980087, 0.031310658901929855, 0.006186673417687416, 0.02889353595674038, -0.0731697529554367, 0.016533030197024345, 0.041661959141492844, 0.02147749252617359, 0.0063850064761936665, -0.02691182866692543, 0.0011352173751220107, -0.08928168565034866, 0.03256421908736229, 0.

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)